In [2]:
import pandas as pd
import numpy as np
import glob
import xgboost as xgb
import requests
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [3]:
# URL to the GitHub folder (NOT raw, NOT the file, but the folder)
api_url = "https://api.github.com/repos/HardyUnine/ML_Project/contents/plan_b"

# Get list of files in the folder
files = requests.get(api_url).json()

# Extract only CSV file URLs
csv_urls = [f["download_url"] for f in files if f["name"].endswith(".csv")]

# Load all CSVs into one dataframe
dfs = []
for url in csv_urls:
    data = pd.read_csv(url)
    data["source_file"] = url.split("/")[-1].replace(".csv", "")  # optional tag
    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)


# df = pd.read_csv("nom_a_changer.csv") # CHANGE CSV NAME HERE ZZ
# df.head()

# # Cant have white space between words when wanting to draw trees!!

In [4]:
df.drop(['DAY', 'TICKER', 'COMPANY_NAME', 'source_file'], axis=1, inplace=True)

df.head()

,TOTAL_SHARES,SIR,RSI,BF,ADV,PUBLIC,PRICE_PER_SHARE,SS
0,500000000,25,50,0.100,30000000,480000000,100.0,0
1,500000000,25,48,0.097,29500000,480000000,98.5,0
2,500000000,25,46,0.094,29000000,480000000,97.0,0
3,500000000,25,44,0.091,28500000,480000000,95.5,0
4,500000000,25,42,0.088,28000000,480000000,94.0,0


In [5]:
# Split data in dependent and independant variables
X = df.drop(columns=["SS"])

print(X.head())

y = df["SS"].copy() # REPLACE NAME ZZ
print(y.head())

   TOTAL_SHARES  SIR  RSI     BF       ADV     PUBLIC  PRICE_PER_SHARE
0     500000000   25   50  0.100  30000000  480000000            100.0
1     500000000   25   48  0.097  29500000  480000000             98.5
2     500000000   25   46  0.094  29000000  480000000             97.0
3     500000000   25   44  0.091  28500000  480000000             95.5
4     500000000   25   42  0.088  28000000  480000000             94.0
0    0
1    0
2    0
3    0
4    0
Name: SS, dtype: int64


In [6]:
print(sum(y)/len(y)) # to check how many positives we have
# sum of how many squeezes, divided by total 
# data will probably be inbalances (ex. 30% squeezes) so need to make sure that there is 30% in both the train and test data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y) # stratify = yes, we want same percetages everywjere

print(sum(y_train)/len(y_train))
print(sum(y_test)/len(y_test)) # check that they have the same percentages of squeezes

0.26071428571428573
0.2619047619047619
0.2571428571428571


In [7]:
# clf_xgb = xgb.XGBClassifier(objective='binary:logistic', missing=None) # creates shell in which we will make the trees
# clf_xgb.fit(X_train, 
#             y_train, 
#             verbose=True, # verbose="tell me what you are doing when you are doing it"
#             early_stopping_rounds=10, # if after 10 trees the prediction is not improving, stop
#             eval_metric="aucpr",
#             eval_set=[(X_test, y_test)]) # it is testing how many trees with need with the test df


# Create classifier with early stopping built-in
clf_xgb = xgb.XGBClassifier(
    objective='binary:logistic',
    missing=None,
    eval_metric="aucpr",
    n_estimators=1000,
    early_stopping_rounds=10  # set early stopping here
)

# Fit model — only X_train, y_train, eval_set, verbose
clf_xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=True)


[0]	validation_0-aucpr:0.77957
[1]	validation_0-aucpr:0.94921
[2]	validation_0-aucpr:0.95439
[3]	validation_0-aucpr:0.95137
[4]	validation_0-aucpr:0.95465
[5]	validation_0-aucpr:0.95621
[6]	validation_0-aucpr:0.95712
[7]	validation_0-aucpr:0.95866
[8]	validation_0-aucpr:0.96303
[9]	validation_0-aucpr:0.96218
[10]	validation_0-aucpr:0.96876
[11]	validation_0-aucpr:0.96772
[12]	validation_0-aucpr:0.97049
[13]	validation_0-aucpr:0.97114
[14]	validation_0-aucpr:0.97209
[15]	validation_0-aucpr:0.96887
[16]	validation_0-aucpr:0.96946
[17]	validation_0-aucpr:0.96916
[18]	validation_0-aucpr:0.96916
[19]	validation_0-aucpr:0.96806
[20]	validation_0-aucpr:0.96643
[21]	validation_0-aucpr:0.96643
[22]	validation_0-aucpr:0.96534
[23]	validation_0-aucpr:0.96534


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,10
,enable_categorical,False
,eval_metric,'aucpr'


In [8]:
import sys
print(sys.executable)  # must be /Users/.../ml-env/bin/python


/Users/angelikiandreadi/.pyenv/versions/ml-env/bin/python


In [9]:
y_pred = clf_xgb.predict(X_test)

#cm = confusion_matrix(y_test, y_pred)

XGBoostError: [13:09:41] /Users/runner/work/xgboost/xgboost/src/c_api/../common/json_utils.h:46: Invalid type for: `missing`, expecting one of the: {``Number`, `Integer`}, got: `Null`
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000000132128428 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x0000000132130b0c void xgboost::TypeCheck<xgboost::JsonNumber, xgboost::JsonInteger>(xgboost::Json const&, xgboost::StringView) + 308
  [bt] (2) 3   libxgboost.dylib                    0x00000001321309a4 xgboost::GetMissing(xgboost::Json const&) + 272
  [bt] (3) 4   libxgboost.dylib                    0x00000001321aa548 InplacePredictImpl(std::__1::shared_ptr<xgboost::DMatrix>, char const*, xgboost::Learner*, unsigned long long const**, unsigned long long*, float const**) + 140
  [bt] (4) 5   libxgboost.dylib                    0x00000001321ab0cc XGBoosterPredictFromColumnar + 420
  [bt] (5) 6   libffi.dylib                  